In [1]:
%matplotlib inline
import nope
import nivapy3 as nivapy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn

sn.set_context('notebook')

# Estimating loads in unmonitored regions - 2018

The new model can be used to estimate loads in unmonitored areas. We know the regine ID for each of the 155 stations where water chemistry is measured, and we also know which OSPAR region each monitoring site drains to. We want to use observed data to estimate loads upstream of each monitoring point, and modelled data elsewhere. This can be achieved using the output from the new model.

This notebook is based on the one [here](http://nbviewer.jupyter.org/github/JamesSample/rid/blob/master/notebooks/loads_unmonitored_regions.ipynb). It first runs the NOPE model for 2018 and then extracts data for unmonitored regions.

In [2]:
# Connect to db
engine = nivapy.da.connect()

Username:  ···
Password:  ········


Connection successful.


## 1. Generate model input file

In [3]:
# Year of interest
year = 2018

# Parameters of interest
par_list = ['Tot-N', 'Tot-P']

# Folder containing NOPE data
nope_fold = r'../../../NOPE/NOPE_Core_Input_Data'

# Ouput path for model file
out_csv = r'../../../NOPE/NOPE_Annual_Inputs/nope_input_data_%s.csv' % year

In [4]:
# Make input file
df = nope.make_rid_input_file(year, engine, nope_fold, out_csv,
                              par_list=par_list)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4244: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  **kwargs


## 2. Run model

In [5]:
%%time
# Input file
in_csv = r'../../../NOPE/NOPE_Annual_Inputs/nope_input_data_%s.csv' % year

# Run model
g = nope.run_nope(in_csv, par_list)

CPU times: user 7.98 s, sys: 90 ms, total: 8.07 s
Wall time: 8.28 s


## 3. Save results

In [6]:
# Save results as csv
out_csv = r'../../../NOPE/nope_results_%s.csv' % year
df = nope.model_to_dataframe(g, out_path=out_csv)

df.head()

,regine,regine_ned,accum_agri_diff_tot-n_tonnes,accum_agri_diff_tot-p_tonnes,accum_agri_pt_tot-n_tonnes,accum_agri_pt_tot-p_tonnes,accum_all_point_tot-n_tonnes,accum_all_point_tot-p_tonnes,accum_all_sources_tot-n_tonnes,accum_all_sources_tot-p_tonnes,...,local_q_reg_m3/s,local_ren_tot-n_tonnes,local_ren_tot-p_tonnes,local_runoff_mm/yr,local_spr_tot-n_tonnes,local_spr_tot-p_tonnes,local_trans_tot-n,local_trans_tot-p,local_urban_tot-n_tonnes,local_urban_tot-p_tonnes
0,315.0,315.,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.358167,0.009445,...,0.018282,0.0000,0.0000,97.717858,0.000000,0.00000,1.00,1.00,0.0,0.0
1,315.,300_315,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.358167,0.009445,...,0.000000,0.0000,0.0000,0.000000,0.000000,0.00000,1.00,1.00,0.0,0.0
2,314.C,314.B,2.281586,0.023028,0.011128,0.000403,0.112128,0.002479,10.521073,0.078933,...,0.550787,0.0000,0.0000,364.754848,0.121687,0.00593,0.83,0.35,0.0,0.0
3,314.B,314.A,10.932857,0.071830,0.053322,0.001257,2.086862,0.034739,41.063607,0.299357,...,0.917020,1.4684,0.0147,319.160492,0.823000,0.11200,0.85,0.26,0.0,0.0
4,314.A,314.5,10.932857,0.071830,0.053322,0.001257,2.086862,0.034739,41.915765,0.309582,...,0.066289,0.0000,0.0000,319.160492,0.000000,0.00000,1.00,1.00,0.0,0.0


In [7]:
# Save version with main catchments only
main_list = ["%03d." % i for i in range(1, 316)]
df2 = df.query('regine in @main_list')
df2.sort_values('regine', inplace=True)

# Save
out_csv = r'../../../NOPE/nope_results_%s_main_catchs.csv' % year
df2.to_csv(out_csv, index=False, encoding='utf-8')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


## 4. Explore results

### 4.1. Total N and P

####  4.1.1. Identify areas with monitoring data

Where observations are available, we want to use them in preference to the model output. This means identifying all the catchments with observed data and substracting the model results for these locations. This is more complicated than it appears, because a small number of observed catchments are upstream of others, so subtracting all the loads for the 155 monitored catchments involves "double accounting", which we want to avoid. The first step is therefore to identify the downstream-most nodes for the monitored areas i.e. for the cases where one catchment is upstream of another, we just want the downstream node.

In [8]:
# Read station data
in_xlsx = r'../../../Data/RID_Sites_List.xlsx'
stn_df = pd.read_excel(in_xlsx, sheet_name='RID_All')

# Get just cols of interest and drop duplicates 
# (some sites are in the same regine)
stn_df = stn_df[['ospar_region', 'nve_vassdrag_nr']].drop_duplicates()

# Get catch IDs with calib data
calib_nds = set(stn_df['nve_vassdrag_nr'].values)

# Build network
in_path = r'../../../NOPE/NOPE_Annual_Inputs/nope_input_data_1990.csv'
g, nd_list = nope.build_calib_network(in_path, calib_nds)

# Get list of downstream nodes
ds_nds = []
for nd in g:
    # If no downstream nodes
    if g.out_degree(nd) == 0:
        # Node is of interest
        ds_nds.append(nd)

# Get just the downstream catchments
stn_df = stn_df[stn_df['nve_vassdrag_nr'].isin(ds_nds)]

#### 4.1.2. Sum model results for monitored locations

In [9]:
# Read model output
in_csv = r'../../../NOPE/nope_results_%s.csv' % year
nope_df = pd.read_csv(in_csv)

# Join accumulated outputs to stns of interest
mon_df = pd.merge(stn_df, nope_df, how='left',
                  left_on='nve_vassdrag_nr',
                  right_on='regine')

# Groupby OSPAR region
mon_df = mon_df.groupby('ospar_region').sum()

# Get just accum cols
cols = [i for i in mon_df.columns if i.split('_')[0]=='accum']
mon_df = mon_df[cols]

mon_df.head()

,accum_agri_diff_tot-n_tonnes,accum_agri_diff_tot-p_tonnes,accum_agri_pt_tot-n_tonnes,accum_agri_pt_tot-p_tonnes,accum_all_point_tot-n_tonnes,accum_all_point_tot-p_tonnes,accum_all_sources_tot-n_tonnes,accum_all_sources_tot-p_tonnes,accum_anth_diff_tot-n_tonnes,accum_anth_diff_tot-p_tonnes,...,accum_nat_diff_tot-n_tonnes,accum_nat_diff_tot-p_tonnes,accum_q_m3/s,accum_ren_tot-n_tonnes,accum_ren_tot-p_tonnes,accum_spr_tot-n_tonnes,accum_spr_tot-p_tonnes,accum_upstr_area_km2,accum_urban_tot-n_tonnes,accum_urban_tot-p_tonnes
ospar_region,,,,,,,,,,,,,,,,,,,,,
LOFOTEN-BARENTS SEA,140.062687,5.306710,2.539554,0.218376,90.979272,7.527509,4337.761091,63.627262,140.062687,5.306710,...,4106.719133,50.793043,910.318100,38.871976,3.005651,49.567741,4.303482,63555.61,0.000000,0.000000
NORTH SEA,2570.953785,67.329914,34.862152,2.176470,400.436149,35.263558,14033.497074,199.676245,2576.926355,67.873391,...,11056.134570,96.539297,1777.289999,207.749206,18.746915,135.663566,11.105319,23353.19,5.972570,0.543476
NORWEGIAN SEA2,2674.734844,71.693221,40.763819,3.061545,567.117603,52.434888,12206.549799,251.902234,2691.201822,73.822372,...,8948.230375,125.644975,1697.030131,290.154869,17.007917,193.190915,20.021126,45896.63,16.466978,2.129151
SKAGERAK,11028.318989,217.282549,102.079627,5.546824,3866.321171,106.056806,32039.568816,487.867552,11190.254228,234.760312,...,16982.993416,147.050434,2359.711355,2701.428472,22.976213,840.286879,49.375668,93945.27,161.935239,17.477763


This table gives the **modelled** inputs to each OSPAR region from catchments for which we have observed data. We want to subtract these values from the overall modelled inputs to each region and substitute the observed data instead.

The trickiest part of this is that the OSPAR regions in the TEOTIL catchment network (and therefore the network for my new model too) don't exactly match the new OSPAR definitions. The OSPAR boundaries were updated relatively recently, so instead of simply selecting the desired OSPAR region in the model output, I need to aggregate based on vassdragsnummers.

**Note:** Eventually, it would be a good idea to update the network information in `regine.csv` to reflect the current OSPAR regions.

#### 4.1.3. Group model output according to "new" OSPAR regions

In [10]:
# Define "new" OSPAR regions
os_dict = {'SKAGERAK':(1, 23),
           'NORTH SEA':(24, 90),
           'NORWEGIAN SEA2':(91, 170),
           'LOFOTEN-BARENTS SEA':(171, 247)}

# Container for results
df_list = []

# Loop over model output
for reg in os_dict.keys():
    min_id, max_id = os_dict[reg]
    
    regs = ['%03d.' % i for i in range(min_id, max_id+1)]
    
    # Get data for this region
    df2 = nope_df[nope_df['regine'].isin(regs)]
    
    # Get just accum cols
    cols = [i for i in df2.columns if i.split('_')[0]=='accum']
    df2 = df2[cols]
    
    # Add region
    df2['ospar_region'] = reg
    
    # Add sum to output
    df_list.append(df2)

# Build df
os_df = pd.concat(df_list, axis=0)

# Aggregate
os_df = os_df.groupby('ospar_region').sum()

os_df.head()

,accum_agri_diff_tot-n_tonnes,accum_agri_diff_tot-p_tonnes,accum_agri_pt_tot-n_tonnes,accum_agri_pt_tot-p_tonnes,accum_all_point_tot-n_tonnes,accum_all_point_tot-p_tonnes,accum_all_sources_tot-n_tonnes,accum_all_sources_tot-p_tonnes,accum_anth_diff_tot-n_tonnes,accum_anth_diff_tot-p_tonnes,...,accum_nat_diff_tot-n_tonnes,accum_nat_diff_tot-p_tonnes,accum_q_m3/s,accum_ren_tot-n_tonnes,accum_ren_tot-p_tonnes,accum_spr_tot-n_tonnes,accum_spr_tot-p_tonnes,accum_upstr_area_km2,accum_urban_tot-n_tonnes,accum_urban_tot-p_tonnes
ospar_region,,,,,,,,,,,,,,,,,,,,,
LOFOTEN-BARENTS SEA,585.600335,27.772068,10.565114,0.885819,16682.430896,2764.598308,26859.471949,2936.503916,595.572645,29.175909,...,9581.468408,142.729698,2391.332871,1325.566415,142.242211,339.448700,41.045803,138090.89,9.972310,1.403841
NORTH SEA,6188.251302,186.887310,80.323926,5.829535,23377.833800,3859.245264,52234.556488,4269.998464,6228.068783,191.955980,...,22628.653904,218.797220,3665.657990,3199.961126,424.717608,694.128141,81.489400,59314.38,39.817482,5.068670
NORWEGIAN SEA2,7305.820077,229.296760,101.708615,8.082202,30749.635915,5093.782998,54869.114175,5574.292742,7337.978868,233.537202,...,16781.499392,246.972542,3396.540099,2451.103988,299.247732,719.280869,87.373479,113934.05,32.158791,4.240442
SKAGERAK,12699.908284,277.179492,112.985724,6.352341,10172.752860,258.832966,41366.589567,728.761481,12973.642753,309.911458,...,18220.193954,160.017056,2479.012365,7829.125980,103.760654,992.979023,65.188249,102574.69,273.734469,32.731966


We can now calculate the unmonitored component by simply subtracting the values modelled upstream of monitoring stations from the overall modelled inputs to each OSPAR region.

#### 4.1.4. Estimate loads in unmonitored areas

In [11]:
# Calc unmonitored loads
unmon_df = os_df - mon_df

# Write output
out_csv = r'../../../NOPE/unmonitored_loads_%s.csv' % year
unmon_df.to_csv(out_csv, encoding='utf-8', index_label='ospar_region')

unmon_df.round(0).astype(int).T

ospar_region,LOFOTEN-BARENTS SEA,NORTH SEA,NORWEGIAN SEA2,SKAGERAK
accum_agri_diff_tot-n_tonnes,446,3617,4631,1672
accum_agri_diff_tot-p_tonnes,22,120,158,60
accum_agri_pt_tot-n_tonnes,8,45,61,11
accum_agri_pt_tot-p_tonnes,1,4,5,1
accum_all_point_tot-n_tonnes,16591,22977,30183,6306
accum_all_point_tot-p_tonnes,2757,3824,5041,153
accum_all_sources_tot-n_tonnes,22522,38201,42663,9327
accum_all_sources_tot-p_tonnes,2873,4070,5322,241
accum_anth_diff_tot-n_tonnes,456,3651,4647,1783
accum_anth_diff_tot-p_tonnes,24,124,160,75


#### 4.1.5. Aggregate values to required quantities

In [12]:
# Aggregate to match report
unmon_df['flow'] = unmon_df['accum_q_m3/s']*60*60*24/1000. # 1000s m3/day

unmon_df['sew_n'] = unmon_df['accum_ren_tot-n_tonnes'] + unmon_df['accum_spr_tot-n_tonnes']
unmon_df['sew_p'] = unmon_df['accum_ren_tot-p_tonnes'] + unmon_df['accum_spr_tot-p_tonnes']

unmon_df['ind_n'] = unmon_df['accum_ind_tot-n_tonnes']
unmon_df['ind_p'] = unmon_df['accum_ind_tot-p_tonnes']

unmon_df['fish_n'] = unmon_df['accum_aqu_tot-n_tonnes']
unmon_df['fish_p'] = unmon_df['accum_aqu_tot-p_tonnes']

unmon_df['diff_n'] = unmon_df['accum_anth_diff_tot-n_tonnes'] + unmon_df['accum_nat_diff_tot-n_tonnes']
unmon_df['diff_p'] = unmon_df['accum_anth_diff_tot-p_tonnes'] + unmon_df['accum_nat_diff_tot-p_tonnes']

new_df = unmon_df[['flow', 'sew_n', 'sew_p', 
                   'ind_n', 'ind_p', 'fish_n', 
                   'fish_p', 'diff_n', 'diff_p']]

# Total for Norway
new_df.loc['NORWAY'] = new_df.sum(axis=0)

# Reorder rows
new_df = new_df.reindex(['NORWAY', 'LOFOTEN-BARENTS SEA', 'NORTH SEA', 
                         'NORWEGIAN SEA2', 'SKAGERAK'])

new_df.round().astype(int)

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,flow,sew_n,sew_p,ind_n,ind_p,fish_n,fish_p,diff_n,diff_p
ospar_region,,,,,,,,,
NORWAY,448260,13095,1099,2582,262,60256,10404,36655,731
LOFOTEN-BARENTS SEA,127960,1577,176,128,6,14879,2574,5930,116
NORTH SEA,163155,3551,476,487,95,18894,3249,15224,246
NORWEGIAN SEA2,146838,2687,350,992,113,26442,4574,12480,281
SKAGERAK,10308,5280,97,975,48,40,7,3021,88


## 5. Other N and P species

Tore's procedure `RESA2.FIXTEOTILPN` defines simple correction factors for estimating PO4, NO3 and NH4 from total P and N. The table below lists the factors used.

|   Source    | Phosphate | Nitrate | Ammonium |
|:-----------:|:---------:|:-------:|:--------:|
|    Sewage   |     0.600 |   0.050 |    0.750 |
|   Industry  |     0.600 |   0.050 |    0.750 |
| Aquaculture |     0.690 |   0.110 |    0.800 |
|   Diffuse   |     0.246 |   0.625 |    0.055 |


In [13]:
# Dict of conversion factors
con_dict = {('sew', 'po4'):('p', 0.6),
            ('ind', 'po4'):('p', 0.6),
            ('fish', 'po4'):('p', 0.69),
            ('diff', 'po4'):('p', 0.246),
            ('sew', 'no3'):('n', 0.05),
            ('ind', 'no3'):('n', 0.05),
            ('fish', 'no3'):('n', 0.11),
            ('diff', 'no3'):('n', 0.625),
            ('sew', 'nh4'):('n', 0.75),
            ('ind', 'nh4'):('n', 0.75),
            ('fish', 'nh4'):('n', 0.8),
            ('diff', 'nh4'):('n', 0.055)}

# Apply factors
for src in ['sew', 'ind', 'fish', 'diff']:
    for spc in ['po4', 'no3', 'nh4']:
        el, fac = con_dict[(src, spc)]
        new_df[src+'_'+spc] = fac * new_df[src+'_'+el]
        
new_df.round().astype(int).T

ospar_region,NORWAY,LOFOTEN-BARENTS SEA,NORTH SEA,NORWEGIAN SEA2,SKAGERAK
flow,448260,127960,163155,146838,10308
sew_n,13095,1577,3551,2687,5280
sew_p,1099,176,476,350,97
ind_n,2582,128,487,992,975
ind_p,262,6,95,113,48
fish_n,60256,14879,18894,26442,40
fish_p,10404,2574,3249,4574,7
diff_n,36655,5930,15224,12480,3021
diff_p,731,116,246,281,88
sew_po4,659,106,286,210,58


## 6. Other quantities

The model currently only considers N and P, but the project focuses on a wider range of parameters. For now, we simply assume that all measured inputs (`renseanlegg`, `industri` and `akvakultur`) for regines outside of catchments with measured data make it to the sea.

We only want data for catchments that are not monitored i.e. for regine IDs **not** in the graph created above.

In [14]:
# The sql below uses a horrible (and slow!) hack to get around Oracle's
# 1000 item limit on IN clauses. See here for details:
# https://stackoverflow.com/a/9084247/505698
nd_list_hack = [(1, i) for i in nd_list]

sql = ("SELECT SUBSTR(a.regine, 1, 3) AS vassdrag, "
       "  a.type, "
       "  b.name, "
       "  b.unit, "
       "  SUM(c.value * d.factor) as value "
       "FROM RESA2.RID_PUNKTKILDER a, "
       "RESA2.RID_PUNKTKILDER_OUTPAR_DEF b, "
       "RESA2.RID_PUNKTKILDER_INPAR_VALUES c, "
       "RESA2.RID_PUNKTKILDER_INP_OUTP d "
       "WHERE a.anlegg_nr = c.anlegg_nr "
       "AND (1, a.regine) NOT IN %s "
       "AND d.in_pid = c.inp_par_id "
       "AND d.out_pid = b.out_pid "
       "AND c.year = %s "
       "GROUP BY SUBSTR(a.regine, 1, 3), a.type, b.name, b.unit "
       "ORDER BY SUBSTR(a.regine, 1, 3), a.type" % (tuple(nd_list_hack), year))

df = pd.read_sql(sql, engine)

# Tidy
df['par'] = df['type'] + '_' + df['name'] + '_' + df['unit']
del df['name'], df['unit'], df['type']

# Pivot
df = df.pivot(index='vassdrag', columns='par', values='value')
df.reset_index(inplace=True)

In [15]:
def f(x):
    try:
        a = int(x)
        return a
    except:
        return -999

# Convert vassdrag to numbers
df['vass'] = df['vassdrag'].apply(f)

# Get just the main catchments
df = df.query('vass != -999')

df.head()

par,vassdrag,INDUSTRI_As_tonn,INDUSTRI_Cd_tonn,INDUSTRI_Cr_tonn,INDUSTRI_Cu_tonn,INDUSTRI_Hg_tonn,INDUSTRI_NH3_tonn,INDUSTRI_NH4-N_tonn,INDUSTRI_Ni_tonn,INDUSTRI_PCB_tonn,...,RENSEANLEGG_Hg_tonn,RENSEANLEGG_Ni_tonn,RENSEANLEGG_PAH_tonn,RENSEANLEGG_PCB_tonn,RENSEANLEGG_Pb_tonn,RENSEANLEGG_S.P.M._tonn,RENSEANLEGG_Tot-N_tonn,RENSEANLEGG_Tot-P_tonn,RENSEANLEGG_Zn_tonn,vass
0,001,0.042070,0.009330,0.009350,0.051450,0.000000e+00,NaN,NaN,0.062560,NaN,...,0.000040,0.011154,0.000007,0.0,0.001142,87.04374,118.889480,1.439065,0.074045,1
1,002,0.015353,0.007239,0.657614,3.564189,6.218900e-03,NaN,NaN,0.350561,NaN,...,0.000347,0.212970,0.000629,0.0,0.015638,380.21215,720.306080,6.874860,0.917181,2
2,003,0.000024,0.000003,0.000065,0.000897,3.000000e-07,NaN,NaN,0.000238,NaN,...,0.000029,0.019094,NaN,NaN,0.001242,131.32459,287.206060,2.894490,0.125102,3
3,004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000016,0.005101,NaN,NaN,0.000711,NaN,210.275385,1.285670,0.045276,4
4,005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,37.587500,0.353110,NaN,5


In [16]:
def f2(x):   
    if x in range(1, 24):
        return 'SKAGERAK'
    elif x in range(24, 91):
        return 'NORTH SEA'
    elif x in range(91, 171):
        return 'NORWEGIAN SEA2'
    elif x in range(171, 248):
        return 'LOFOTEN-BARENTS SEA'
    else:
        return np.nan

# Assign main catchments to OSPAR regions
df['osp_reg'] = df['vass'].apply(f2)

df.head()

par,vassdrag,INDUSTRI_As_tonn,INDUSTRI_Cd_tonn,INDUSTRI_Cr_tonn,INDUSTRI_Cu_tonn,INDUSTRI_Hg_tonn,INDUSTRI_NH3_tonn,INDUSTRI_NH4-N_tonn,INDUSTRI_Ni_tonn,INDUSTRI_PCB_tonn,...,RENSEANLEGG_Ni_tonn,RENSEANLEGG_PAH_tonn,RENSEANLEGG_PCB_tonn,RENSEANLEGG_Pb_tonn,RENSEANLEGG_S.P.M._tonn,RENSEANLEGG_Tot-N_tonn,RENSEANLEGG_Tot-P_tonn,RENSEANLEGG_Zn_tonn,vass,osp_reg
0,001,0.042070,0.009330,0.009350,0.051450,0.000000e+00,NaN,NaN,0.062560,NaN,...,0.011154,0.000007,0.0,0.001142,87.04374,118.889480,1.439065,0.074045,1,SKAGERAK
1,002,0.015353,0.007239,0.657614,3.564189,6.218900e-03,NaN,NaN,0.350561,NaN,...,0.212970,0.000629,0.0,0.015638,380.21215,720.306080,6.874860,0.917181,2,SKAGERAK
2,003,0.000024,0.000003,0.000065,0.000897,3.000000e-07,NaN,NaN,0.000238,NaN,...,0.019094,NaN,NaN,0.001242,131.32459,287.206060,2.894490,0.125102,3,SKAGERAK
3,004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.005101,NaN,NaN,0.000711,NaN,210.275385,1.285670,0.045276,4,SKAGERAK
4,005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,37.587500,0.353110,NaN,5,SKAGERAK


In [17]:
# Group by OSPAR region
df.fillna(0, inplace=True)
df = df.groupby('osp_reg').sum()
df.drop(0, inplace=True)

# Total for Norway
df.loc['NORWAY'] = df.sum(axis=0)

# Join to model results 
df = new_df.join(df)

# Get cols of interest
umod_cols = ['S.P.M.', 'TOC', 'As', 'Pb', 'Cd', 'Cu', 'Zn', 'Ni', 'Cr', 'Hg']
umod_cols = ['%s_%s_tonn' % (i, j) for i in ['INDUSTRI', 'RENSEANLEGG'] for j in umod_cols]
cols = list(new_df.columns) + umod_cols
cols.remove('RENSEANLEGG_TOC_tonn')
df = df[cols]

df.round(0).astype(int).T

ospar_region,NORWAY,LOFOTEN-BARENTS SEA,NORTH SEA,NORWEGIAN SEA2,SKAGERAK
flow,448260,127960,163155,146838,10308
sew_n,13095,1577,3551,2687,5280
sew_p,1099,176,476,350,97
ind_n,2582,128,487,992,975
ind_p,262,6,95,113,48
fish_n,60256,14879,18894,26442,40
fish_p,10404,2574,3249,4574,7
diff_n,36655,5930,15224,12480,3021
diff_p,731,116,246,281,88
sew_po4,659,106,286,210,58


## 7. Fish farm copper

Finally, we need to add in the Cu totals from fish farms. The method is similar to that used above, but simpler because we're only interested in one parameter.

In [18]:
# The sql below uses a horrible (and slow!) hack to get around Oracle's
# 1000 item limit on IN clauses. See here for details:
# https://stackoverflow.com/a/9084247/505698
nd_list_hack = [(1, i) for i in nd_list]

sql = ("SELECT SUBSTR(a.regine, 1, 3) AS vassdrag, "
       "  SUM(b.value) as value "
       "FROM RESA2.RID_KILDER_AQUAKULTUR a, "
       "RESA2.RID_KILDER_AQKULT_VALUES b "
       "WHERE a.nr = b.anlegg_nr "
       "AND (1, a.regine) NOT IN %s "
       "AND b.inp_par_id = 41 "
       "AND b.ar = %s "
       "GROUP BY SUBSTR(a.regine, 1, 3), b.inp_par_id "
       "ORDER BY SUBSTR(a.regine, 1, 3), b.inp_par_id" % (tuple(nd_list_hack), year))

aq_df = pd.read_sql(sql, engine)

# Get vassdrag
aq_df['vass'] = aq_df['vassdrag'].apply(f)
aq_df = aq_df.query('vass != -999')

# Calc OSPAR region and group
aq_df['osp_reg'] = aq_df['vass'].apply(f2)
aq_df.fillna(0, inplace=True)
aq_df = aq_df.groupby('osp_reg').sum()
del aq_df['vass']

# Total for Norway
aq_df.loc['NORWAY'] = aq_df.sum(axis=0)

# Rename
aq_df.columns = ['AQUAKULTUR_Cu_tonn',]

# Join model results 
df = df.join(aq_df)

df.round(0).astype(int).T

ospar_region,NORWAY,LOFOTEN-BARENTS SEA,NORTH SEA,NORWEGIAN SEA2,SKAGERAK
flow,448260,127960,163155,146838,10308
sew_n,13095,1577,3551,2687,5280
sew_p,1099,176,476,350,97
ind_n,2582,128,487,992,975
ind_p,262,6,95,113,48
fish_n,60256,14879,18894,26442,40
fish_p,10404,2574,3249,4574,7
diff_n,36655,5930,15224,12480,3021
diff_p,731,116,246,281,88
sew_po4,659,106,286,210,58


In [19]:
# Write output
out_csv = r'../../../Results/Unmon_loads/unmon_loads_%s.csv' % year
df.to_csv(out_csv)

This data can then be used to create Table 3 in the report - see [this notebook](https://nbviewer.jupyter.org/github/JamesSample/rid/blob/master/notebooks/summary_table_2017.ipynb) for details.